In [1]:
"""
Conveyor Belt Color Detection System using YOLOv8
With Enhanced Linux Camera Support
------------------------------------------------
This script uses YOLOv8 to detect objects on a conveyor belt,
classify them by color, and control the conveyor system accordingly.
Includes specialized Linux camera handling to work with external webcams.
"""

import cv2
import numpy as np
import serial
import time
import subprocess
import os
import platform
import threading
from ultralytics import YOLO

# Import Linux Camera Manager class from the previous file
# You can also directly include the class here
class LinuxCameraManager:
    """Specialized camera manager for Linux systems"""
    
    @staticmethod
    def find_available_cameras():
        """Find all available camera devices on the system"""
        print("Searching for camera devices...")
        
        # Check for standard device files
        device_paths = []
        try:
            # List all video devices
            for i in range(10):
                device_path = f"/dev/video{i}"
                if os.path.exists(device_path):
                    device_paths.append(device_path)
        except Exception as e:
            print(f"Error listing device files: {e}")
        
        if device_paths:
            print(f"Found {len(device_paths)} video device(s):")
            for path in device_paths:
                print(f"- {path}")
        else:
            print("No video devices found in /dev/")
        
        return device_paths
    
    @staticmethod
    def test_open_device(device_path, timeout=5):
        """
        Test if a camera device can be opened and read from.
        
        Args:
            device_path: Path to the camera device
            timeout: Maximum time to wait for a valid frame
            
        Returns:
            Tuple of (success, width, height, fps) or None if failed
        """
        print(f"Testing camera device: {device_path}")
        
        # Try to open the camera
        try:
            cap = cv2.VideoCapture(device_path)
            
            if not cap.isOpened():
                print(f"Failed to open {device_path}")
                return None
            
            # Try to read frames with timeout
            start_time = time.time()
            while (time.time() - start_time) < timeout:
                ret, frame = cap.read()
                if ret and frame is not None and frame.size > 0:
                    # Successfully read a frame
                    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    fps = cap.get(cv2.CAP_PROP_FPS)
                    
                    print(f"Successfully opened {device_path}: {width}x{height} @ {fps}fps")
                    cap.release()
                    return (True, width, height, fps)
                
                time.sleep(0.1)
            
            print(f"Timeout waiting for valid frame from {device_path}")
            cap.release()
            return None
            
        except Exception as e:
            print(f"Error testing {device_path}: {e}")
            return None
    
    @staticmethod
    def initialize_with_v4l2(device_path, width=640, height=480):
        """
        Initialize camera using direct V4L2 settings
        This can help when standard OpenCV settings don't work
        
        Args:
            device_path: Path to the camera device
            width: Desired width
            height: Desired height
            
        Returns:
            Camera capture object or None if failed
        """
        # First, try to set V4L2 parameters directly using v4l2-ctl
        try:
            # Set video format
            subprocess.run([
                "v4l2-ctl", 
                "--device", device_path,
                "--set-fmt-video=width=640,height=480,pixelformat=YUYV"
            ], check=False)
            
            # Set framerate
            subprocess.run([
                "v4l2-ctl", 
                "--device", device_path,
                "--set-parm=30"
            ], check=False)
            
            print(f"Applied V4L2 settings to {device_path}")
        except Exception as e:
            print(f"Warning: Failed to apply V4L2 settings: {e}")
        
        # Now open with OpenCV
        try:
            cap = cv2.VideoCapture(device_path)
            if not cap.isOpened():
                print(f"Failed to open {device_path} after V4L2 setup")
                return None
            
            # Set properties
            cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
            
            # Test if it works
            ret, frame = cap.read()
            if not ret or frame is None:
                print(f"Failed to read frame from {device_path} after setup")
                cap.release()
                return None
            
            return cap
            
        except Exception as e:
            print(f"Error in V4L2 initialization: {e}")
            return None
    
    @staticmethod
    def try_gstreamer_pipeline(device_path="/dev/video0", width=640, height=480):
        """
        Try to initialize camera using GStreamer pipeline
        This is helpful on systems like Raspberry Pi
        
        Args:
            device_path: Camera device path
            width: Desired width
            height: Desired height
            
        Returns:
            Camera capture object or None if failed
        """
        # Check if OpenCV has GStreamer support
        if cv2.getBuildInformation().find("GStreamer") == -1:
            print("OpenCV not built with GStreamer support")
            return None
        
        try:
            # Create GStreamer pipeline
            pipeline = (
                f"v4l2src device={device_path} ! "
                f"video/x-raw, width={width}, height={height} ! "
                "videoconvert ! appsink"
            )
            
            print(f"Trying GStreamer pipeline: {pipeline}")
            
            cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
            if not cap.isOpened():
                print("Failed to open camera with GStreamer pipeline")
                return None
            
            # Test if it works
            ret, frame = cap.read()
            if not ret or frame is None:
                print("Failed to read frame with GStreamer pipeline")
                cap.release()
                return None
            
            print("Successfully initialized camera with GStreamer")
            return cap
            
        except Exception as e:
            print(f"GStreamer error: {e}")
            return None
    
    @classmethod
    def get_best_camera(cls):
        """
        Find and initialize the best available camera using multiple methods
        
        Returns:
            Tuple of (camera_capture, device_path) or (None, None) if failed
        """
        # Find available cameras
        device_paths = cls.find_available_cameras()
        
        if not device_paths:
            print("No camera devices found!")
            return None, None
        
        # Test each device and select the best one
        for device_path in device_paths:
            # First try standard OpenCV capture
            result = cls.test_open_device(device_path)
            if result:
                success, width, height, fps = result
                
                # Reopen the camera for actual use
                cap = cv2.VideoCapture(device_path)
                if cap.isOpened():
                    print(f"Using camera: {device_path}")
                    return cap, device_path
            
            # If standard method failed, try V4L2 direct setup
            print(f"Standard initialization failed for {device_path}, trying V4L2 method...")
            cap = cls.initialize_with_v4l2(device_path)
            if cap is not None:
                print(f"Using camera with V4L2 initialization: {device_path}")
                return cap, device_path
        
        # If all direct methods failed, try GStreamer with the first device
        if device_paths:
            print("All direct methods failed, trying GStreamer...")
            cap = cls.try_gstreamer_pipeline(device_paths[0])
            if cap is not None:
                print(f"Using camera with GStreamer: {device_paths[0]}")
                return cap, f"gstreamer:{device_paths[0]}"
        
        print("Failed to initialize any camera device")
        return None, None


class ConveyorColorDetectionSystem:
    def __init__(self, camera=None, serial_port='/dev/ttyUSB0', baud_rate=9600):
        """
        Initialize the conveyor color detection system.
        
        Args:
            camera: Pre-initialized camera object (for Linux systems)
            serial_port: Serial port for Arduino/microcontroller communication
            baud_rate: Baud rate for serial communication
        """
        # System components
        self.camera = camera  # Can be pre-initialized for Linux
        self.camera_id = None  # Only used if camera not pre-initialized
        self.serial_connection = None
        self.serial_port = serial_port
        self.baud_rate = baud_rate
        
        # YOLO model
        self.model = None
        
        # System state
        self.running = False
        self.detected_objects = []
        self.conveyor_speed = 50  # Initial speed (0-100%)
        
        # Color classification parameters
        self.color_ranges = {
            'red': ([0, 100, 100], [10, 255, 255]),
            'blue': ([100, 100, 100], [130, 255, 255]),
            'green': ([40, 100, 100], [80, 255, 255]),
            'yellow': ([20, 100, 100], [35, 255, 255])
        }
        
    def initialize_system(self):
        """Initialize camera, model, and serial connection."""
        try:
            # Initialize camera if not pre-initialized
            if self.camera is None:
                print("No pre-initialized camera provided, attempting to initialize...")
                if platform.system() == "Linux":
                    # Use Linux-specific camera handling
                    self.camera, device_path = LinuxCameraManager.get_best_camera()
                    if self.camera is None:
                        print("Failed to initialize camera on Linux.")
                        return False
                    print(f"Camera initialized on Linux: {device_path}")
                else:
                    # Standard camera initialization for non-Linux systems
                    self.camera = cv2.VideoCapture(0)  # Default to first camera
                    if not self.camera.isOpened():
                        print("Failed to open camera")
                        return False
                        
            # Initialize YOLO model
            print("Initializing YOLOv8 model...")
            try:
                self.model = YOLO("yolov8n.pt")  # Load standard YOLOv8 nano model
                print("YOLOv8 model loaded successfully")
            except Exception as e:
                print(f"Error loading YOLO model: {e}")
                print("Make sure ultralytics package is installed (pip install ultralytics)")
                return False
            
            # Initialize serial connection to microcontroller
            try:
                # Determine appropriate serial port based on OS
                if platform.system() == "Windows":
                    default_ports = ["COM3", "COM4", "COM5"]
                else:  # Linux/Mac
                    default_ports = ["/dev/ttyUSB0", "/dev/ttyACM0"]
                
                # Try the specified port first, then try default ports
                try_ports = [self.serial_port] + default_ports
                
                for port in try_ports:
                    try:
                        self.serial_connection = serial.Serial(port, self.baud_rate, timeout=1)
                        print(f"Connected to microcontroller on {port}")
                        break
                    except Exception:
                        continue
                        
                if self.serial_connection is None:
                    print("Warning: Could not connect to microcontroller")
                    print("Running in camera-only mode (no conveyor control)")
            except Exception as e:
                print(f"Serial connection error: {e}")
                print("Running in camera-only mode (no conveyor control)")
            
            print("System initialization complete")
            return True
            
        except Exception as e:
            print(f"Initialization error: {e}")
            self.cleanup()
            return False
            
    def detect_objects(self, frame):
        """
        Detect objects in the frame using YOLOv8.
        
        Args:
            frame: Input image frame
            
        Returns:
            List of detected objects with bounding boxes
        """
        try:
            results = self.model(frame)
            return results[0].boxes  # Return bounding boxes
        except Exception as e:
            print(f"Error in object detection: {e}")
            return []
        
    def determine_color(self, frame, box):
        """
        Determine the dominant color of an object.
        
        Args:
            frame: Input image frame
            box: Bounding box of the object
            
        Returns:
            String representing the detected color
        """
        try:
            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Ensure coordinates are within frame boundaries
            height, width = frame.shape[:2]
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(width - 1, x2)
            y2 = min(height - 1, y2)
            
            # Skip if box is invalid
            if x1 >= x2 or y1 >= y2:
                return "unknown"
            
            # Extract the object region
            object_region = frame[y1:y2, x1:x2]
            
            # Convert to HSV color space
            hsv_region = cv2.cvtColor(object_region, cv2.COLOR_BGR2HSV)
            
            # Calculate the average HSV values
            avg_hsv = np.mean(hsv_region, axis=(0, 1))
            
            # Check each color range
            for color_name, (lower, upper) in self.color_ranges.items():
                lower = np.array(lower)
                upper = np.array(upper)
                
                if (avg_hsv[0] >= lower[0] and avg_hsv[0] <= upper[0] and
                    avg_hsv[1] >= lower[1] and avg_hsv[1] <= upper[1] and
                    avg_hsv[2] >= lower[2] and avg_hsv[2] <= upper[2]):
                    return color_name
                    
            return "unknown"
        except Exception as e:
            print(f"Error in color determination: {e}")
            return "unknown"
        
    def control_conveyor(self, command):
        """
        Send control command to the conveyor system.
        
        Args:
            command: Command string to send to the microcontroller
        """
        if self.serial_connection:
            try:
                self.serial_connection.write(f"{command}\n".encode())
            except Exception as e:
                print(f"Error sending command to conveyor: {e}")
            
    def set_conveyor_speed(self, speed):
        """
        Set the conveyor belt speed.
        
        Args:
            speed: Speed value (0-100%)
        """
        if speed < 0:
            speed = 0
        elif speed > 100:
            speed = 100
            
        self.conveyor_speed = speed
        self.control_conveyor(f"SPEED:{speed}")
        
    def sort_object(self, color):
        """
        Trigger sorting mechanism based on detected color.
        
        Args:
            color: Detected color string
        """
        self.control_conveyor(f"SORT:{color}")
        
    def process_frame(self, frame):
        """
        Process a video frame for object detection and color classification.
        
        Args:
            frame: Input video frame
            
        Returns:
            Processed frame with annotations
        """
        if frame is None:
            print("Warning: Received empty frame")
            return None
            
        # Create a copy of the frame to avoid modification issues
        frame_copy = frame.copy()
        
        # Reset detected objects list
        self.detected_objects = []
        
        try:
            # Detect objects
            boxes = self.detect_objects(frame_copy)
            
            # Process each detected object
            for box in boxes:
                # Get object class and confidence
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                
                # Only process if confidence is high enough
                if conf > 0.5:
                    # Determine the object's color
                    color_name = self.determine_color(frame_copy, box)
                    
                    # Extract coordinates
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    
                    # Add to detected objects list
                    self.detected_objects.append({
                        'class': cls,
                        'class_name': self.model.names[cls],
                        'color': color_name,
                        'confidence': conf,
                        'box': (x1, y1, x2, y2)
                    })
                    
                    # Draw bounding box
                    cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    # Add label with class name and color
                    label = f"{self.model.names[cls]}: {color_name} ({conf:.2f})"
                    cv2.putText(frame_copy, label, (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    # Trigger sorting mechanism if object reaches certain position
                    if x2 > frame_copy.shape[1] * 0.8:  # If object is at the right side of the frame
                        self.sort_object(color_name)
        
        except Exception as e:
            print(f"Error processing frame: {e}")
            # Add error text to frame
            cv2.putText(frame_copy, f"Error: {str(e)}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
        # Add system info to the frame
        cv2.putText(frame_copy, f"Conveyor Speed: {self.conveyor_speed}%", (10, frame_copy.shape[0] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
        return frame_copy
        
    def run(self):
        """Run the main detection and control loop."""
        if not self.initialize_system():
            return
            
        self.running = True
        
        if self.serial_connection:
            self.set_conveyor_speed(self.conveyor_speed)
        
        frame_count = 0
        last_fps_time = time.time()
        fps = 0
        
        try:
            while self.running:
                # Capture frame with error handling
                try:
                    ret, frame = self.camera.read()
                    if not ret or frame is None:
                        print("Failed to capture frame - trying again")
                        time.sleep(0.1)  # Short delay
                        continue
                        
                    # Process frame
                    processed_frame = self.process_frame(frame)
                    if processed_frame is None:
                        continue
                    
                    # Calculate FPS
                    frame_count += 1
                    if frame_count % 10 == 0:
                        current_time = time.time()
                        fps = 10 / (current_time - last_fps_time)
                        last_fps_time = current_time
                    
                    # Add FPS to the frame
                    cv2.putText(processed_frame, f"FPS: {fps:.1f}", (10, 30), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    
                    # Display the processed frame
                    cv2.imshow("Conveyor Color Detection", processed_frame)
                    
                except Exception as e:
                    print(f"Error in main loop: {e}")
                    time.sleep(0.1)  # Short delay to prevent rapid error loops
                
                # Break the loop on 'q' key press
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('+') or key == ord('='):
                    # Increase speed
                    self.set_conveyor_speed(min(100, self.conveyor_speed + 10))
                elif key == ord('-'):
                    # Decrease speed
                    self.set_conveyor_speed(max(0, self.conveyor_speed - 10))
                elif key == ord('c'):
                    # Capture a still frame for color calibration
                    if frame is not None:
                        self.calibrate_colors(frame)
                    
        finally:
            self.cleanup()
            
    def calibrate_colors(self, frame):
        """
        Interactive color calibration tool.
        Allows user to click on objects to define color ranges.
        
        Args:
            frame: Current video frame
        """
        calibration_frame = frame.copy()
        hsv_frame = cv2.cvtColor(calibration_frame, cv2.COLOR_BGR2HSV)
        
        # Create window for calibration
        cv2.namedWindow("Color Calibration")
        
        # Store clicked points
        clicked_points = []
        
        # Mouse callback function
        def mouse_callback(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDOWN:
                # Get RGB and HSV values at clicked point
                b, g, r = calibration_frame[y, x]
                h, s, v = hsv_frame[y, x]
                
                # Add point to list
                clicked_points.append((x, y, (r, g, b), (h, s, v)))
                
                # Draw circle at clicked point
                cv2.circle(calibration_frame, (x, y), 5, (0, 255, 0), -1)
                
                # Show values
                print(f"Point at ({x}, {y}):")
                print(f"  RGB: ({r}, {g}, {b})")
                print(f"  HSV: ({h}, {s}, {v})")
                
                # Update display
                cv2.imshow("Color Calibration", calibration_frame)
        
        # Set mouse callback
        cv2.setMouseCallback("Color Calibration", mouse_callback)
        
        # Instructions
        print("\nCOLOR CALIBRATION")
        print("=================")
        print("Click on objects to sample colors")
        print("Press 's' to save a new color range")
        print("Press 'q' to quit calibration")
        
        # Show initial frame
        cv2.imshow("Color Calibration", calibration_frame)
        
        while True:
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('s') and clicked_points:
                # Calculate average and range from clicked points
                h_values = [p[3][0] for p in clicked_points]
                s_values = [p[3][1] for p in clicked_points]
                v_values = [p[3][2] for p in clicked_points]
                
                # Calculate ranges (with padding)
                h_min = max(0, min(h_values) - 10)
                h_max = min(179, max(h_values) + 10)
                s_min = max(0, min(s_values) - 20)
                s_max = min(255, max(s_values) + 20)
                v_min = max(0, min(v_values) - 20)
                v_max = min(255, max(v_values) + 20)
                
                # Ask for color name
                color_name = input("Enter name for this color: ")
                
                if color_name:
                    # Add or update color range
                    self.color_ranges[color_name] = (
                        [h_min, s_min, v_min],
                        [h_max, s_max, v_max]
                    )
                    
                    print(f"Added color range for '{color_name}':")
                    print(f"  H: {h_min} to {h_max}")
                    print(f"  S: {s_min} to {s_max}")
                    print(f"  V: {v_min} to {v_max}")
                    
                    # Print all color ranges
                    print("\nCurrent color ranges:")
                    for name, (lower, upper) in self.color_ranges.items():
                        print(f"  '{name}': ({lower}, {upper}),")
                
                # Clear clicked points
                clicked_points = []
                calibration_frame = frame.copy()
                cv2.imshow("Color Calibration", calibration_frame)
                
            elif key == ord('q'):
                break
        
        cv2.destroyWindow("Color Calibration")
            
    def cleanup(self):
        """Clean up resources."""
        self.running = False
        
        if self.camera is not None:
            self.camera.release()
            
        if self.serial_connection is not None:
            self.control_conveyor("STOP")
            self.serial_connection.close()
            
        cv2.destroyAllWindows()


class ArduinoController:
    """
    Arduino sketch for controlling the conveyor belt and sorting mechanism.
    This is provided as a reference for the microcontroller code needed.
    """
    
    ARDUINO_CODE = """
    // Conveyor Belt Controller for Color Detection System
    
    // Pin definitions
    const int MOTOR_PIN_1 = 9;    // PWM pin for motor control
    const int MOTOR_PIN_2 = 10;   // Direction control
    
    // Servo pins for sorting mechanism
    const int RED_SERVO_PIN = 3;
    const int BLUE_SERVO_PIN = 5;
    const int GREEN_SERVO_PIN = 6;
    const int YELLOW_SERVO_PIN = 9;
    
    #include <Servo.h>
    
    Servo redServo;
    Servo blueServo;
    Servo greenServo;
    Servo yellowServo;
    
    int conveyorSpeed = 0;  // Speed value (0-255)
    
    void setup() {
      // Initialize serial communication
      Serial.begin(9600);
      
      // Initialize motor control pins
      pinMode(MOTOR_PIN_1, OUTPUT);
      pinMode(MOTOR_PIN_2, OUTPUT);
      
      // Initialize servos
      redServo.attach(RED_SERVO_PIN);
      blueServo.attach(BLUE_SERVO_PIN);
      greenServo.attach(GREEN_SERVO_PIN);
      yellowServo.attach(YELLOW_SERVO_PIN);
      
      // Set initial positions
      redServo.write(90);
      blueServo.write(90);
      greenServo.write(90);
      yellowServo.write(90);
      
      // Start conveyor at zero speed
      setConveyorSpeed(0);
      
      Serial.println("Conveyor control initialized");
    }
    
    void loop() {
      // Check for commands from the computer
      if (Serial.available() > 0) {
        String command = Serial.readStringUntil('\n');
        processCommand(command);
      }
    }
    
    void processCommand(String command) {
      // Parse the command
      if (command.startsWith("SPEED:")) {
        // Extract speed value
        int speed = command.substring(6).toInt();
        setConveyorSpeed(speed);
        Serial.print("Speed set to: ");
        Serial.println(speed);
      }
      else if (command.startsWith("SORT:")) {
        // Extract color
        String color = command.substring(5);
        triggerSorter(color);
        Serial.print("Sorting ");
        Serial.println(color);
      }
      else if (command == "STOP") {
        setConveyorSpeed(0);
        Serial.println("Emergency stop");
      }
    }
    
    void setConveyorSpeed(int speed) {
      // Convert 0-100% to 0-255 PWM
      conveyorSpeed = map(speed, 0, 100, 0, 255);
      
      // Set motor speed and direction
      analogWrite(MOTOR_PIN_1, conveyorSpeed);
      digitalWrite(MOTOR_PIN_2, HIGH);  // Set direction
    }
    
    void triggerSorter(String color) {
      // Activate the appropriate servo based on color
      if (color == "red") {
        redServo.write(45);  // Move to sorting position
        delay(500);          // Wait for object to fall
        redServo.write(90);  // Return to neutral
      }
      else if (color == "blue") {
        blueServo.write(45);
        delay(500);
        blueServo.write(90);
      }
      else if (color == "green") {
        greenServo.write(45);
        delay(500);
        greenServo.write(90);
      }
      else if (color == "yellow") {
        yellowServo.write(45);
        delay(500);
        yellowServo.write(90);
      }
    }
    """


def main():
    """Main function to start the system."""
    print("YOLOv8 Conveyor Color Detection System")
    print("--------------------------------------")
    
    # Check if running on Linux
    if platform.system() == "Linux":
        print("Running on Linux - using specialized camera handling")
        
        # Get camera using Linux-specific manager
        camera, device_path = LinuxCameraManager.get_best_camera()
        
        if camera is None:
            print("Failed to initialize camera. Exiting.")
            print("\nTroubleshooting tips:")
            print("1. Check camera connections")
            print("2. Make sure you have permission to access the camera:")
            print("   sudo usermod -a -G video $USER")
            print("3. Install v4l-utils: sudo apt-get install v4l-utils")
            print("4. Try loading the UVC driver: sudo modprobe uvcvideo")
            print("5. If using Raspberry Pi camera module, enable it in raspi-config")
            return
        
        print(f"Camera initialized: {device_path}")
        
        # Create and run the system with pre-initialized camera
        system = ConveyorColorDetectionSystem(
            camera=camera,
            serial_port='/dev/ttyUSB0',
            baud_rate=9600
        )
    else:
        # Standard initialization for non-Linux systems
        system = ConveyorColorDetectionSystem(
            camera=None,  # Will initialize camera internally
            serial_port='COM3' if platform.system() == "Windows" else '/dev/ttyUSB0',
            baud_rate=9600
        )
    
    # Run the system
    
    system.run()


def print_usage():
    """Print usage instructions"""
    print("\nYOLOv8 Conveyor Color Detection System")
    print("--------------------------------------")
    print("Controls:")
    print("  q - Quit the application")
    print("  + - Increase conveyor speed")
    print("  - - Decrease conveyor speed")
    print("  c - Calibrate colors (click on objects to define color ranges)")
    print("\nArduino Setup:")
    print("  Upload the Arduino sketch to your microcontroller")
    print("  Connect the microcontroller to USB")
    print("  Connect the motor driver to pins 9 and 10")
    print("  Connect servos to pins 3, 5, 6, and 9 for sorting")


if __name__ == "__main__":
    print_usage()
    
    
    main()


YOLOv8 Conveyor Color Detection System
--------------------------------------
Controls:
  q - Quit the application
  + - Increase conveyor speed
  - - Decrease conveyor speed
  c - Calibrate colors (click on objects to define color ranges)

Arduino Setup:
  Upload the Arduino sketch to your microcontroller
  Connect the microcontroller to USB
  Connect the motor driver to pins 9 and 10
  Connect servos to pins 3, 5, 6, and 9 for sorting
YOLOv8 Conveyor Color Detection System
--------------------------------------
No pre-initialized camera provided, attempting to initialize...
Initializing YOLOv8 model...
YOLOv8 model loaded successfully
Running in camera-only mode (no conveyor control)
System initialization complete



c:\Users\91861\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 480x640 1 person, 269.2ms
Speed: 15.2ms preprocess, 269.2ms inference, 21.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 186.2ms
Speed: 5.5ms preprocess, 186.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 163.1ms
Speed: 5.0ms preprocess, 163.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 154.5ms
Speed: 2.2ms preprocess, 154.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 142.7ms
Speed: 3.0ms preprocess, 142.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 147.4ms
Speed: 3.0ms preprocess, 147.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 140.2ms
Speed: 3.0ms preprocess, 140.2ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 186.5ms
Spee